## Create Script That Converts Image into a normalized grayscale tensor, with features extracted.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import torch
from torchvision import transforms
from PIL import Image
import random
from sklearn.utils import resample


In [12]:
# Load OpenCV DNN
model_path = "C:/Users/ahmad/Desktop/EngagementML/opencv-dnn/res10_300x300_ssd_iter_140000.caffemodel"
config_path = "C:/Users/ahmad/Desktop/EngagementML/opencv-dnn/deploy.prototxt"


# Define a PyTorch transform for grayscale conversion
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to 1-channel grayscale
    transforms.ToTensor(),                         # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.75], std=[0.75])   # Normalize: shift and scale pixel values
])


def preprocess_frame(image_path, face_net, target_size=(80, 80)):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image {image_path}")
        return

    # Prepare the image for the DNN
    blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(300, 300),
                                 mean=(104.0, 177.0, 123.0), swapRB=False, crop=False)
    face_net.setInput(blob)
    detections = face_net.forward()

    (h, w) = image.shape[:2]
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Confidence threshold
            # Get the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")

            # Ensure the bounding box is within the image dimensions
            x, y, x1, y1 = max(0, x), max(0, y), min(w, x1), min(h, y1)

            # Crop the face
            cropped_face = image[y:y1, x:x1]

            resized_face = cv2.resize(cropped_face, target_size)

            # Convert the resized face to a PIL image
            pil_face = Image.fromarray(cv2.cvtColor(resized_face, cv2.COLOR_BGR2RGB))

            transformed_face_tensor = transform(pil_face)

            return transformed_face_tensor


# Load the pre-trained DNN model
face_net = cv2.dnn.readNetFromCaffe(config_path, model_path)


img_path = r'C:\Users\ahmad\Desktop\EngagementML\resized_faces\51004210183.jpg_face0.jpg'

preprocess_frame(img_path, face_net, target_size=(80, 80))



tensor([[[-0.0118,  0.0144,  0.0092,  ..., -0.4353, -0.1163, -0.0588],
         [ 0.0039, -0.0222, -0.0222,  ..., -0.5399, -0.2993, -0.0850],
         [-0.0013, -0.0118, -0.0170,  ..., -0.6288, -0.4824, -0.0954],
         ...,
         [-0.6497, -0.7542, -0.7908,  ..., -0.7961, -0.7647, -0.7438],
         [-0.7961, -0.7961, -0.8065,  ..., -0.7961, -0.7595, -0.7281],
         [-0.8327, -0.7961, -0.8065,  ..., -0.8013, -0.7647, -0.7281]]])

## Create Dataset Class

In [ ]:
class FacesDataset(Dataset):
    def __init__(self, data_dir, label_csv, transform=None, num_frames=10):
        """
        Args:
            data_dir (str): Path to the directory containing video frames organized in folders.
            label_csv (str): Path to the CSV file containing ClipID and labels.
            transform (callable, optional): Optional transformations to apply to frames.
            num_frames (int): Number of frames per video to include as features.
        """
        self.data_dir = data_dir
        self.labels = self.load_labels(label_csv)
        self.transform = transform
        self.num_frames = num_frames


    def load_labels(self, label_csv):
        """
        Load labels from the CSV file and return a dictionary of ClipID to label mappings.
        """
        label_df = pd.read_csv(label_csv)
        df_map = {}
        for _, row in label_df.iterrows():
            df_map[row['ClipID']] = row['Boredom']
        return df_map

    def get_file_paths(self, clip_id):
        """
        Get sorted file paths for frames in a given clip folder.
        """
        clip_folder = os.path.join(self.data_dir, f"{clip_id}_frames")
        frame_files = sorted(os.listdir(clip_folder))
        return [os.path.join(clip_folder, frame) for frame in frame_files[:self.num_frames]]
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        """
        Retrieve a single sample (frames and label) based on the index.
        """
        clip_id = list(self.labels.keys())[idx]
        label = self.labels[clip_id]

        # Get file paths for the frames
        frame_paths = self.get_file_paths(clip_id)

        # Load and preprocess the frames
        frames = []
        for frame_path in frame_paths:
            processed_frame = preprocess_frame(frame_path, self.face_net, target_size=self.target_size)
            if processed_frame is not None:
                frames.append(processed_frame)

        # Stack all frames into a single tensor
        frames_tensor = torch.stack(frames) if frames else torch.zeros(self.num_frames, 1, *self.target_size)

        return frames_tensor, torch.tensor(label, dtype=torch.long)



## Create Convolutional Neural Network

In [ ]:
def create_model():
    pass


## Train Model

In [ ]:
def train_model():
    pass

## Main Method

In [ ]:
def main():
    pass

if __name__ == '__main__':
    main()